In [41]:
%matplotlib widget

In [42]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [43]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [44]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [45]:
file_parameters['watershed'] = 'han_auto'

In [46]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [47]:
pd.set_option('display.max_columns', 1000)

In [48]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
model_input_path = file_parameters['watershed']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [49]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [50]:
gain_epochs , rnn_epochs

(2000, 15)

# run file

In [51]:
run_num = range(len(folder))


# run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [52]:
__GAIN_TRAINING__ = False

In [53]:
month=12
day=31
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times , month, day= make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], 
                               interpolation=interpolation_option[idx],first_file_no=i, month=month, day=day)

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/의암호_2015.xlsx
data/han/자동/의암호_2016.xlsx
data/han/자동/의암호_2017.xlsx
data/han/자동/의암호_2018.xlsx
data/han/자동/의암호_2019.xlsx
data/han/자동/의암호_2020.xlsx
time ran

In [54]:
print(real_df_all.shape)

(48936, 36)


In [55]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

In [56]:
df[0]

,tmpr_value,ph_value,do_value,ec_value,toc_value,총질소_값,총인_값,클로로필-a_값,Day sin,Day cos,Year sin,Year cos
0,-1.394369,-1.612812,-0.119654,0.532520,-0.615219,1.139064,-0.435205,-0.146837,1.745186e-12,1.414209,-0.073841,1.425860
1,-1.393647,-1.611973,-0.065378,0.569188,-0.746527,1.115699,-0.435205,-0.137743,3.660242e-01,1.366021,-0.072818,1.425858
2,-1.414522,-1.621867,-0.199313,0.569188,-0.576917,1.083570,-0.315723,-0.144563,7.071044e-01,1.224741,-0.071795,1.425856
3,-1.434211,-1.637111,-0.170782,0.568545,-0.571392,1.077510,-0.291251,-0.144563,9.999966e-01,0.999997,-0.070772,1.425853
4,-1.417834,-1.628580,-0.129853,0.562755,-0.633367,1.069371,-0.338756,-0.133196,1.224741e+00,0.707104,-0.069749,1.425848
...,...,...,...,...,...,...,...,...,...,...,...,...
48931,1.226862,-0.067114,-0.745836,0.479127,0.829283,0.553501,0.716426,-0.065792,-1.366021e+00,0.366024,-0.781181,-1.202284
48932,1.214644,-0.255041,-0.885503,0.491993,0.838967,0.573735,0.716426,-0.152668,-1.224741e+00,0.707104,-0.782071,-1.201786
48933,1.195763,-0.416744,-1.008506,0.491993,0.712952,0.563518,0.608461,-0.121245,-9.999966e-01,0.999997,-0.782959,-1.201287
48934,1.190211,-0.453176,-1.052154,0.491993,0.640246,0.675782,0.572473,0.029402,-7.071044e-01,1.224741,-0.783848,-1.200788


# slice DF

In [57]:
# fake_df=real_df_all.iloc[:-2000, :]

In [107]:
train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)

In [108]:
val_df.shape, test_df.shape, train_df.shape

((9788, 36), (4893, 36), (34255, 36))

In [109]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (34255, 36) val_df.shape :  (9788, 36) test_df.shape: (4893, 36)


# target column

In [126]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "tn"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  tn
target_col_idx :  5
out_num_features :  1


In [127]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

# if __RNN_TRAINING__:
#     if not os.path.exists('save/' + model_input_path):
#         os.makedirs('save/' + model_input_path)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + model_input_path+ '/' + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/han/models/han_auto/nitrogen/


In [128]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
plt.figure()
plt.plot(test_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [130]:
__RNN_TRAINING__ = True
continue_train = True

In [131]:
from tensorflow.keras.callbacks import Callback

In [132]:
class checkpoint_save(Callback):
    def __init__(self, model=None, save_path=None, val_nse=None):
        super(Callback, self).__init__()        
#         print('val_nse in checkpoint = ' , val_nse)
        self.best_score = val_nse
        self.model = model
        self.save_path=save_path

    def on_epoch_end(self, epoch, logs={}):
        #current_score = logs.get('val_accuracy')
#         current_score = logs.get('accuracy')
        current_score = logs.get('val_nse')
#         print('current_score = ',current_score)
#         print('self.best_score = ', float(val_nse[2]))
#         print(type(float(val_nse[2])))
#         print('current_score = ', current_score)
        
#         print(type(current_score))
        if current_score > self.best_score:
            print('*****save model: nse from %.3f to %.3f*****' %(self.best_score, current_score) )
            self.best_score = current_score            
            self.model.save_weights(self.save_path)
            

In [133]:
def compile_and_fit(model, window, patience=1000, epochs=400, save_path=None, val_nse=-100):
    
    print('val_nse= ', val_nse)

    checkpoint = checkpoint_save(model=model, save_path=save_path, val_nse = val_nse)
    
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         save_path, monitor='val_nse', verbose=1,
#         save_best_only=True, 
#         save_weights_only= True,
#         mode='max', period=1)

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min')

    model.compile(
        loss=tf.losses.MeanSquaredError(),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.MeanAbsoluteError(), nse])

    history = model.fit(
        #window.train, epochs=epochs,
        window.train, epochs=epochs, steps_per_epoch=10,
        validation_data=window.val,
        callbacks=[early_stopping, checkpoint])
    return history

In [134]:
def model_gru(OUT_STEPS=24*5, out_num_features=1, window=None, epochs=100, training_flag=False, 
              checkpoint_path="save/", continue_train = False):
    model = GRUModel(OUT_STEPS, out_num_features)
    #checkpoint_path = "save/gru_model.ckpt"
#   model.load_weights(checkpoint_path)
    if training_flag == True:
        model_evaluate_value = 0
        if continue_train :
            print("continue")
            model.load_weights(checkpoint_path)
            compile(model)
            model_evaluate_value = model.evaluate(window.val)
            print('model_evaluate_value = ', model_evaluate_value)
            history = compile_and_fit(model,window, epochs=epochs, save_path = checkpoint_path,
                                      val_nse=model_evaluate_value[2])
            
        else:
            history = compile_and_fit(model,window, epochs=epochs, save_path = checkpoint_path)                
        model.load_weights(checkpoint_path)
    else:
        model.load_weights(checkpoint_path)
        compile(model)
    return model

In [ ]:
rnn_epochs = 15

# print('Linear Running....')
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt", continue_train=continue_train)

# print('ELMAN Running....')
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt", continue_train=continue_train)

print('GRU Running....')
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=continue_train)

# print('LSTM Running....')
# multi_lstm_model = model_multi_lstm(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt", continue_train=continue_train)

# print('CNN Running....')
# multi_conv_model = model_multi_conv(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt", continue_train=continue_train)


GRU Running....
continue
73/73 [==============================] - 8s 104ms/step - loss: 0.1317 - mean_absolute_error: 0.2633 - nse: 0.7234
model_evaluate_value =  [0.13370303809642792, 0.26439839601516724, 0.7186229228973389]
val_nse=  0.7186229228973389
Epoch 1/15
10/10 [==============================] - 13s 1s/step - loss: 0.1990 - mean_absolute_error: 0.3108 - nse: 0.8094 - val_loss: 0.2019 - val_mean_absolute_error: 0.3494 - val_nse: 0.5743
Epoch 2/15
10/10 [==============================] - 10s 1s/step - loss: 0.2037 - mean_absolute_error: 0.3135 - nse: 0.7924 - val_loss: 0.1421 - val_mean_absolute_error: 0.2772 - val_nse: 0.7015
Epoch 3/15
10/10 [==============================] - 11s 1s/step - loss: 0.1843 - mean_absolute_error: 0.2997 - nse: 0.8147 - val_loss: 0.1714 - val_mean_absolute_error: 0.3081 - val_nse: 0.6405
Epoch 4/15
10/10 [==============================] - 11s 1s/step - loss: 0.2038 - mean_absolute_error: 0.2966 - nse: 0.7844 - val_loss: 0.2217 - val_mean_absolute_e

## core / window.py / 

In [140]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return float(nse3), float(np.abs(pbias3)), pred_day, labels
    

In [141]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
34255,-0.589025,-1.222362,-0.318497,-0.588743,0.281309,0.327294,-0.230791,-0.046384,1.366021e+00,-0.366024,-0.855492,1.197595,-1.015508,-0.476479,0.415560,-1.302801,-0.988377,-0.885802,0.572473,-0.674020,1.366021e+00,-0.366024,-0.855492,1.197595,-0.554595,-1.350554,-0.155813,-1.477134,0.053738,0.976915,-1.011021,-0.628456,1.366021e+00,-0.366024,-0.855492,1.197595
34256,-0.600132,-1.239844,-0.322463,-0.588743,0.265081,0.313922,-0.531654,-0.038990,1.224741e+00,-0.707104,-0.854635,1.198145,-1.034389,-0.476479,0.447305,-1.373564,-0.964152,-0.898323,0.572473,-0.663668,1.224741e+00,-0.707104,-0.854635,1.198145,-0.585693,-1.263147,-0.084391,-1.612226,0.033904,1.094614,-1.011021,-0.598050,1.224741e+00,-0.707104,-0.854635,1.198145
34257,-0.607906,-1.229635,-0.302622,-0.540496,0.168614,0.295566,-0.255263,-0.068565,9.999966e-01,-0.999997,-0.853777,1.198694,-1.003292,-0.447337,0.594114,-1.399295,-0.988377,-0.845108,0.488979,-0.654057,9.999966e-01,-0.999997,-0.853777,1.198694,-0.619012,-1.175739,-0.020907,-1.782699,0.029512,0.884885,-1.011021,-0.622818,9.999966e-01,-0.999997,-0.853777,1.198694
34258,-0.593468,-1.212153,-0.310559,-0.642137,0.331709,0.363806,-0.147297,-0.000912,7.071044e-01,-1.224741,-0.852920,1.199243,-0.844471,-0.199694,0.788538,-1.341399,-0.164381,-0.617224,0.356542,-0.450082,7.071044e-01,-1.224741,-0.852920,1.199243,-0.622344,-1.175739,-0.005032,-1.823870,0.005257,0.756769,-1.011021,-0.635572,7.071044e-01,-1.224741,-0.852920,1.199243
34259,-0.582361,-1.191770,-0.306593,-0.663366,0.313794,0.342520,-0.338756,-0.011263,3.660242e-01,-1.366021,-0.852061,1.199792,-0.677876,-0.126849,0.530630,-1.637314,-0.212862,-0.605754,0.284565,-0.374574,3.660242e-01,-1.366021,-0.852061,1.199792,-0.617902,-1.175739,0.034645,-1.769190,-0.164381,0.695616,-1.011021,-0.645781,3.660242e-01,-1.366021,-0.852061,1.199792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39143,0.621566,-1.175739,-0.628385,-0.743134,0.834110,-0.790641,-1.058526,-0.273741,-3.660242e-01,1.366021,0.229609,-1.353319,0.451639,0.499566,-0.466101,0.569188,-0.079547,-1.575089,-0.291251,-0.574666,-3.660242e-01,1.366021,0.229609,-1.353319,0.723745,-0.330805,-0.103437,0.250757,0.300940,-1.114938,-0.243746,0.043542,-3.660242e-01,1.366021,0.229609,-1.353319
39144,0.627120,-1.175739,-0.625210,-0.743134,0.727494,-0.858255,-1.154975,-0.305164,-2.648262e-12,1.414209,0.228609,-1.353534,0.436090,0.324751,-0.635132,0.541527,0.109489,-1.532518,-0.302768,-0.408955,-2.648262e-12,1.414209,0.228609,-1.353534,0.668213,-0.505620,-0.150256,0.187714,0.201565,-0.932756,-0.194802,-0.099175,-2.648262e-12,1.414209,0.228609,-1.353534
39145,0.619344,-1.175739,-0.609340,-0.743134,0.783216,-0.736174,-0.939044,-0.347679,3.660242e-01,1.366021,0.227610,-1.353749,0.431648,0.222781,-0.793846,0.528017,-0.052908,-1.553378,-0.410733,-0.390285,3.660242e-01,1.366021,0.227610,-1.353749,0.681540,-0.476479,-0.161369,0.196720,0.136129,-1.056089,-0.243746,-0.140839,3.660242e-01,1.366021,0.227610,-1.353749
39146,0.618234,-1.175739,-0.638704,-0.743134,0.574810,-0.799206,-1.154975,-0.386496,7.071044e-01,1.224741,0.226610,-1.353963,0.429426,0.164516,-0.958115,0.569188,-0.031096,-1.625174,-0.147297,-0.361040,7.071044e-01,1.224741,0.226610,-1.353963,0.719301,-0.389071,-0.166526,0.269413,0.211278,-1.091148,-0.291251,-0.120599,7.071044e-01,1.224741,0.226610,-1.353963


In [ ]:
o1 = 1.6690453
nse3 = [0.58094627]
label_day[predict_day].shape = (5, 1)
mean = 1.6657085
0.5809462666511536
3.46138858795166

In [142]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['GRU'])
print(val_pbias['GRU'])

(9788, 36)
5
(396, 168, 36)
(396, 120, 1)
(396, 120, 1)
predictions.shape = (396, 120, 1)
pred_day.shape = (396, 5, 1)


/home/cjinw/work/tf-2/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.6690453
nse3 = [0.58094627]
label_day[predict_day].shape = (5, 1)
mean = 1.6657085
0.5809462666511536
3.46138858795166


In [124]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(34255, 36)
5
(1416, 168, 36)
(1416, 120, 1)
(1416, 120, 1)
predictions.shape = (1416, 120, 1)
pred_day.shape = (1416, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.8421203
nse3 = [0.8064006]
label_day[predict_day].shape = (5, 1)
mean = 1.84313


0.8064005970954895
0.49361273646354675


In [125]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(48936, 36)
5
(2027, 168, 36)
(2027, 120, 1)
(2027, 120, 1)
predictions.shape = (2027, 120, 1)
pred_day.shape = (2027, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 1.792529
nse3 = [0.78800744]
label_day[predict_day].shape = (5, 1)
mean = 1.7935247


0.7880074381828308
0.4300128221511841


In [ ]:
val_nse['LSTM'], val_pbias['LSTM'], pred, label = compa(model=elman_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['LSTM'])
print(val_pbias['LSTM'])

In [ ]:
val_nse['CNN'], val_pbias['CNN'], pred, label = compa(model=multi_conv_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['CNN'])
print(val_pbias['CNN'])

In [ ]:
x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '['+start_year+','+end_year+']' + rnn_target_column)
# plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(val_nse.keys(), val_nse.values(), 0.3, label='NSE')
# plt.xticks(x,val_nse.keys(), rotation=45)
_ = plt.legend()
plt.show()